In [37]:
import os
import re
import sys
sys.path.append('../')

from utils import CharsetMapper

charset = CharsetMapper('../data/charset_36.txt', 26)

data_dir = '../data/IC15'
ic15_training = '../data/ch4_training_word_images_gt'
ic15_test = '../data/ch4_test_word_images_gt'
ic15_training_gt = os.path.join(ic15_training, 'gt.txt')
ic15_test_gt = os.path.join(ic15_test, 'gt.txt')

def convert_ic15_gt_file(gt_path: str, output_path: str, image_dir: str):
    with open(gt_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()
        pattern = re.compile(r'^(.*?), "(.*?)"$')
        results = []
        for line in lines:
            match = pattern.match(line)
            if match:
                img, text = match.group(1), match.group(2)
                text = text.lower()
                charset_values = charset.label_to_char.values()
                if not all([c in charset_values for c in text]):
                    continue
                # # remove characters not in charset
                # text = ''.join([c for c in text.lower() if c in charset_values])
                # if len(text) == 0:
                #     continue
                results.append((img, text))
            else:
                raise Exception("Wrong format")
            
    with open(output_path, 'w', encoding='utf-8') as f:
        for img, text in results:
            f.write(f'{image_dir}{img}\t{text}\n')

convert_ic15_gt_file(ic15_training_gt, os.path.join(data_dir, 'train.txt'), 'train/')
convert_ic15_gt_file(ic15_test_gt, os.path.join(data_dir, 'test.txt'), 'test/')

lines = []
with open(os.path.join(data_dir, 'train.txt'), 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())
    print(len(lines))

with open(os.path.join(data_dir, 'test.txt'), 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())
    print(len(lines))

with open(os.path.join(data_dir, 'labels.txt'), 'w', encoding='utf-8') as f:
    f.writelines(lines)


genaxis theatre
[06]
62-03
nothing?
gen is
$500
$4.00
$4.00
$4.00
!947
minute;
100,000
day;
36,000,000
#ilovesciencecentre
simge.edu.sg
whole-grain
children's
one-north
7-9pm
11:1
ratio.
less?
singapore's
no.1
there's
10%
$10
more!
internet/
top-up
cashcard/flashpay/
now!
#14-01
#14-03
1-5
$1.00
$7.90
$20
3 for
$20.00
pillow.
non-fiction
20%
year!
it's
a...
mia!
side-gate
6-3388-680
#05-11
www.tk-hair.com
experts!
$9.90
$128
$5,000
(usp)
1-12
#10-12
#10-10
#10-15
24/7
$24
season's
m1-a1
by:
recycle!
recycle!
world's
group's
(cgdb)
cgdb,
(us$1,57
billion)
launch.
6,000
home.
life!
www.imob.sg
10%
me,
life,
#03-01/02/03
good!
$128
world's
we're
10-10 to 11
10-12 to 14
10-15 to 16
$10
day.
sexy!
don't
pte.
ltd.
$24
singapore's
3.3m
reader\"
side-gate
residents,
\"scan\"
\"card
gate.
mcdonald's
levi's
out!
to:
\"we
claims.
lives.\"
introducing..
www.ite.edu.sg
we´re
s2.30
$2..30
$2.00
shop&dine
19 99
182/182
x-ing
here!
#05-11
experts!
4-hr
jobstreet.com
shell.
introducucing...
joy!
toast?

In [38]:
import os
import re
import sys
sys.path.append('../')

from utils import CharsetMapper

charset = CharsetMapper('../data/charset_36.txt', 26)

data_dir = '../data/IIIT5k'
iiit5k_train_org = os.path.join(data_dir, 'train_org.txt')
iiit5k_test_org = os.path.join(data_dir, 'test_org.txt')

def filter_iiit5k(gt_path: str, output_path: str):
    with open(gt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        results = []
        for line in lines:
            img, text = line.strip().split()
            text = text.lower()
            charset_values = charset.label_to_char.values()
            if not all([c in charset_values for c in text.lower()]):
                continue
            # # remove characters not in charset
            # text = ''.join([c for c in text if c in charset_values])
            # if len(text) == 0:
            #     continue
            results.append((img, text))
            
    with open(output_path, 'w', encoding='utf-8') as f:
        for img, text in results:
            f.write(f'{img}\t{text}\n')

filter_iiit5k(iiit5k_train_org, os.path.join(data_dir, 'train.txt'))
filter_iiit5k(iiit5k_test_org, os.path.join(data_dir, 'test.txt'))

lines = []
with open(os.path.join(data_dir, 'train.txt'), 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())
    print(len(lines))

with open(os.path.join(data_dir, 'test.txt'), 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())
    print(len(lines))

with open(os.path.join(data_dir, 'labels.txt'), 'w', encoding='utf-8') as f:
    f.writelines(lines)


2000
5000


In [ ]:
def filter_line(line, data_dir, max_length, chars):
    parts = line.strip().split()
    assert len(parts) == 2
    img_file, _ = parts
    label = img_file.split('_')[1]
    img_path = os.path.join(data_dir, img_file)
    if not os.path.exists(img_path):
        return None
    try:
        Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"Error opening image {img_file}: {e}")
        return None
    if len(label) <= max_length and (not chars or any(c in label for c in chars)):
        return line
    return None

def filter_by_length(input_file, output_file, max_length, data_dir=None, chars=None):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        lines = infile.readlines()
        filtered_lines = []
        
        # Use ThreadPoolExecutor to filter lines in parallel
        with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
            future_to_line = {executor.submit(filter_line, line, data_dir, max_length, chars): line for line in lines}
            for future in tqdm(concurrent.futures.as_completed(future_to_line), total=len(lines)):
                filtered_line = future.result()
                if filtered_line:
                    filtered_lines.append(filtered_line)

        # for line in tqdm(lines):
        #     parts = line.strip().split()
        #     assert len(parts) == 2
        #     img_file, _ = parts
        #     label = img_file.split('_')[1]
        #     if data_dir and not os.path.exists(os.path.join(data_dir, img_file)):
        #         continue
        #     try:
        #         Image.open(os.path.join(data_dir, img_file))
        #     except Exception as e:
        #         print(f"Error opening image {img_file}: {e}")
        #         continue
        #     if len(label) <= max_length:
        #         if chars and not any(c in label for c in chars):
        #             continue
        #         filtered_lines.append(line)
        outfile.writelines(filtered_lines)
        print(f"Filtered {len(lines)} lines to {len(filtered_lines)} lines")

In [ ]:

filter_by_length('data/mnt/ramdisk/max/90kDICT32px/annotation_train.txt', 'data/mnt/ramdisk/max/90kDICT32px/annotation_train_ml13.txt', 13, 'data/mnt/ramdisk/max/90kDICT32px')
filter_by_length('data/mnt/ramdisk/max/90kDICT32px/annotation_test.txt', 'data/mnt/ramdisk/max/90kDICT32px/annotation_test_ml13.txt', 13, 'data/mnt/ramdisk/max/90kDICT32px')


In [3]:
with open('../data/mnt/ramdisk/max/90kDICT32px/annotation_test_ml13.txt', 'r') as f:
    lines = f.readlines()
    print(len(lines))

with open('../data/mnt/ramdisk/max/90kDICT32px/annotation_test_ml13_clean.txt', 'r') as f:
    lines = f.readlines()
    print(len(lines))

872818
872817
